In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Reading TFIDF Avg w2v vectors(100-D)

In [2]:
tfidf_avg_w2v =pd.read_pickle("\\Users\\AKSHAY\\Desktop\\ML python course\\Amazon Project\\Logistic Regression\\W2V Vectors\\TFIDF Avg W2V Vectors\\tfidf_avg_vec_100")
print(tfidf_avg_w2v.shape)
tfidf_avg_w2v.head()

(122109, 101)


,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.001005,-0.000993,-0.000176,-0.000016,-0.000637,0.001346,0.000667,0.001510,-0.000133,-0.000779,...,0.001775,-0.000025,-0.001377,0.000550,-0.000228,-0.001434,-0.000971,0.000378,-0.001934,1
1,0.000447,0.000047,-0.000051,0.000002,0.000638,0.000663,-0.001048,-0.000657,0.000902,-0.000835,...,-0.000874,-0.000076,-0.000138,0.000227,0.000570,-0.000014,-0.000653,0.000828,-0.000387,1
2,0.002199,0.000842,-0.001726,-0.000084,-0.000796,0.000935,0.000353,0.000058,0.001372,-0.000609,...,-0.000939,0.000222,0.000223,-0.000879,-0.001259,-0.001296,0.000505,-0.000455,0.001263,0
3,0.000479,-0.000181,0.000898,0.000729,-0.001107,0.000479,-0.000218,-0.000634,-0.000136,-0.000369,...,-0.000357,0.000199,0.000379,0.001250,-0.000416,0.000148,0.000612,-0.000012,0.000545,1
4,0.001807,0.000264,-0.001362,-0.001132,-0.001522,0.000012,0.001637,-0.000874,0.000608,0.000235,...,0.001261,0.000399,-0.000921,0.001763,-0.000563,-0.001746,0.001239,0.001081,0.000124,1


In [4]:
x =tfidf_avg_w2v.iloc[:,:100]
y =tfidf_avg_w2v.iloc[:,100]

In [5]:
print(x.shape)
y.shape

(122109, 100)


(122109,)

## Column Standardization

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
a =StandardScaler()
x =a.fit_transform(x)

In [8]:
x_train,x_test,y_train,y_test =train_test_split(x,y,test_size =0.3)

In [9]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
y_test.shape

(85476, 100)
(36633, 100)
(85476,)


(36633,)

In [10]:
tuned_params = [{'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}]

In [11]:
model = GridSearchCV(LogisticRegression(), tuned_params, scoring = 'accuracy')
model.fit(x_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [13]:
print(model.best_estimator_)
print(model.score(x_test, y_test))

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.649387164578


In [14]:
y_pred = model.best_estimator_.predict(x_test)

In [15]:
acc = accuracy_score(y_test, y_pred, normalize=True) * float(100)
acc

64.938716457838566

## Confusion Matrix

In [16]:
confusion_matrix(y_test,y_pred).T

array([[ 9809,  5503],
       [ 7341, 13980]], dtype=int64)

## L1 regularization and sparsity

In [17]:
clf = LogisticRegression(C=1, penalty = 'l1')
clf.fit(x_train, y_train)
w = clf.coef_    
print(np.count_nonzero(w))

100


This means 100 out of 100 are non zero.That means for regularization takes place.

In [18]:
clf = LogisticRegression(C=0.1, penalty = 'l1')
clf.fit(x_train, y_train)
w = clf.coef_
print(np.count_nonzero(w))

99


As we can see clearly as c decreases sparsity increases.This time 99 features out of 100  are non zero,means 1 features is removed.

In [19]:
clf = LogisticRegression(C=0.001, penalty = 'l1')
clf.fit(x_train, y_train)
w = clf.coef_
print(np.count_nonzero(w))

45


In [20]:
clf = LogisticRegression(C=0.00025, penalty = 'l1')
clf.fit(x_train, y_train)
w = clf.coef_
print(np.count_nonzero(w))

2


At c=0.00025 we are getting top 2 features and rest becomes zero.